In [8]:
import pandas as pd
import json
import geopandas as gpd

In [15]:
census_2021 = pd.read_csv('../data/SA2-T02_Selected_Medians_and_Averages.csv/abs_2021census_t02_aust_sa2-8898358042715570011.csv')


In [17]:
census_2021.columns = ['Average_household_size', 'Median_total_household_income_weekly', 
                      'Average_number_of_persons_per_bedroom', 'Median_age_of_persons',
                      'Median_rent_weekly', 'Median_mortgage_repayment_monthly',
                      'Median_total_personal_income_weekly', 'Median_total_family_income_weekly',
                      'sa2_code_2021']

In [18]:
GCCSA = gpd.read_file('../data/GCCSA_2021_AUST_SHP_GDA2020/GCCSA_2021_AUST_GDA2020.shp')
GCCSA = GCCSA.to_crs('EPSG:7855')
Greater_Melbourne = GCCSA.iloc[4]['geometry']

SA2 = gpd.read_file('../data/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')
SA2 = SA2.to_crs('EPSG:7855')

In [19]:
SA2_GreaterMelbourne = SA2[SA2.within(Greater_Melbourne)]
SA2_GreaterMelbourne = SA2_GreaterMelbourne[['SA2_CODE21', 'SA2_NAME21', 'geometry']]
SA2_GreaterMelbourne['SA2_CODE21'] = SA2_GreaterMelbourne['SA2_CODE21'].astype('int64')

In [24]:
census_2021_geometry = pd.merge(census_2021, SA2_GreaterMelbourne, 
                               how='inner', left_on='sa2_code_2021', right_on='SA2_CODE21').drop('sa2_code_2021', axis=1)

In [25]:
census_2021_geometry = gpd.GeoDataFrame(census_2021_geometry, geometry='geometry')
census_2021_geometry.to_csv('../output/preprocessing/census_SA2_GreaterMelbourne_2021.csv', index=False)
